In [1]:
from humpback import *

from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers, models

## Fetch Data

In [2]:
symbol = 'BTCUSDT'

train_size, val_size, test_size = 0.5, 0.2, 0.3

batch_size = 64
input_sequence_length = 20
forecast_horizon = 1
multi_horizon = False

In [3]:
data_raw_df = pd.read_csv(f'Binance Data/{symbol}.csv', index_col='index')

In [4]:
data_return = getReturn(data_raw_df, 'Close')
data_feature = featureGeneration(data_return)
X_data_df , y_data_df  = getXy(data_feature, 'Return')

/Users/taryn.tsui/Documents/Alvin/GitHub/alvinlui99/Algo Trading/humpback.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,f'SMA_{window}'] = value
/Users/taryn.tsui/Documents/Alvin/GitHub/alvinlui99/Algo Trading/humpback.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,f'SMA_{window}'] = value
/Users/taryn.tsui/Documents/Alvin/GitHub/alvinlui99/Algo Trading/humpback.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [ ]:
X_train_df, X_val_df, X_test_df, X_scaler = preprocess(X_data_df, train_size, val_size, test_size, True)
y_train_df, y_val_df, y_test_df, y_scaler = preprocess(y_data_df, train_size, val_size, test_size, True)

In [ ]:
train_dataset = keras.utils.timeseries_dataset_from_array(
        X_train_df,
        y_train_df,
        sequence_length=input_sequence_length,
        shuffle=False,
    )

In [ ]:
val_dataset = keras.utils.timeseries_dataset_from_array(
        X_val_df,
        y_val_df,
        sequence_length=input_sequence_length,
        shuffle=False,
    ) 

In [ ]:
test_dataset = keras.utils.timeseries_dataset_from_array(
        X_test_df,
        None,
        sequence_length=input_sequence_length,
        shuffle=False,
    ) 

## Model

In [ ]:
class LSTM_stack(layers.Layer):
    """stacks of LSTM with dropout, follwed by dense layer"""

    def __init__(
        self,
        dropout_rate: float = 0.2
        ):
        super().__init__()

        self.lstm_1 = layers.LSTM(
            units=50,
            return_sequences=True,
            )

        self.lstm_2 = layers.LSTM(
            units=50,
            return_sequences=True,
            )

        self.lstm_3 = layers.LSTM(
            units=50,
            return_sequences=False,
            )

        self.dense_1 = layers.Dense(
            units=1
            )

        self.dropout_1 = layers.Dropout(dropout_rate)

    def call(self, inputs):
        x = self.lstm_1(inputs)
        x = self.dropout_1(x)
        x = self.lstm_2(inputs)
        x = self.dropout_1(x)
        x = self.lstm_3(inputs)
        x = self.dropout_1(x)
        return self.dense_1(x)

In [ ]:
inputs  = keras.Input((
    input_sequence_length,
    X_train_df.shape[1]))
outputs = LSTM_stack()(inputs)
model   = keras.Model(inputs, outputs)

In [ ]:
optimizer = keras.optimizers.Adam()
loss      = keras.losses.MeanSquaredError()

model.compile(optimizer=optimizer,loss=loss)

In [ ]:
model.fit(
    train_dataset,
    epochs=20,
    validation_data=val_dataset)

In [ ]:
predictions = pd.DataFrame(model.predict(test_dataset), index=y_test_df.iloc[input_sequence_length-1:,].index)

In [ ]:
plt.plot(y_scaler.inverse_transform(y_test_df), color='red', label='Actual')
plt.plot(y_scaler.inverse_transform(predictions), color='blue', label='Predicted')
plt.title('LSTM BTC 1h')
plt.legend() 